In [1]:
import pandas 

In [2]:
data=pandas.read_csv(r"D:\Estudiar\MLData\CC GENERAL.csv")

In [5]:
data.drop(columns="CUST_ID", inplace=True)

In [6]:
data[data.isnull().any(axis=1)].shape

(314, 17)

In [7]:
#Se imputan los valores a 0

In [8]:
data.fillna(value=0, inplace=True)

In [9]:
#lo bueno de usar hbdscan para la deteccion de anomalias es que no asigna un cluster a todos los puntos, sino aquellos puntos que estan mas separados del resto se etiquetan automaticamente como valores extremos

In [10]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

In [11]:
#Al usar aalgoritmo de clustering basado en densidad tenemos que estandarizar los datos

In [12]:
data_norm=pandas.DataFrame(StandardScaler().fit_transform(data))

In [17]:
clusterer=DBSCAN()
cluster_labels=clusterer.fit_predict(data_norm)
pandas.Series(cluster_labels).value_counts(); #muestra los valores anomalos como -1

In [18]:
from sklearn.metrics import silhouette_score

In [19]:
silhouette_score(data_norm,cluster_labels)

-0.46596190778573116

In [20]:
#un coeficiente negativo significa que es muy malo

In [22]:
from scipy.stats import randint 
from scipy.stats import uniform

distribucion_param={
    "eps": uniform(0,5),
    "min_samples": randint(2,20),
    "p": randint(1,3),
}

In [27]:
import numpy
from sklearn.model_selection import ParameterSampler

n_muestras=10 #
n_iteraciones=3 #para validar, se va a entrenar para cada seleccion de hiperparametros en 3 muestras
pct_muestra=0.7 #usamos el 70% de los datos para entrenar el modelo en cada iteracion
resultados_bus=[]
lista_param=list(ParameterSampler(distribucion_param, n_iter=n_muestras))

for param in lista_param:
    for iteration in range(n_iteraciones):
        param_resultados=[]
        muestra=data_norm.sample(frac=pct_muestra)
        etiquetas_clusters=DBSCAN(n_jobs=2, **param).fit_predict(muestra)
        try:
            param_resultados.append(silhouette_score(muestra, etiquetas_clusters))
        except ValueError: #a veces silhoutte falla en los casos en los que solo hay 
            pass 
    puntuacion_media=numpy.mean(param_resultados)
    resultados_bus.append([puntuacion_media, param])

In [29]:
sorted(resultados_bus, key=lambda x: x[0], reverse=True)[:5]

[[0.7428011755923594, {'eps': 4.8582500753506945, 'min_samples': 14, 'p': 1}],
 [0.7297140601234327, {'eps': 4.818225746329852, 'min_samples': 14, 'p': 1}],
 [0.6782866586236247, {'eps': 4.013825612384533, 'min_samples': 15, 'p': 1}],
 [0.5871650933093374, {'eps': 3.006367767293657, 'min_samples': 9, 'p': 2}],
 [0.23605266325194202, {'eps': 1.3171072956789742, 'min_samples': 17, 'p': 2}]]

In [30]:
mejores_params={"eps": 4.8582500753506945, "min_samples": 14, "p": 1}
clusterer=DBSCAN(n_jobs=2, **mejores_params)
etiquetas_clusters=clusterer.fit_predict(data_norm)

In [31]:
pandas.Series(etiquetas_clusters).value_counts()

 0    8876
-1      74
dtype: int64

In [34]:
#data[(etiquetas_clusters==-1)].describe()

In [56]:
def resumen_cluster(cluster_id):
    cluster=data[etiquetas_clusters==cluster_id]
    resumen_cluster=cluster.mean().to_dict()
    resumen_cluster["cluster_id"]=cluster_id
    return resumen_cluster

def comparar_clusters(cluster_ids):
    resumenes=[]
    for cluster_id in cluster_ids:
        resumenes.append(resumen_cluster(cluster_id))
    return pandas.DataFrame(resumenes).set_index("cluster_id").T

In [57]:
comparar_clusters([0,-1])

cluster_id,0,-1
BALANCE,1526.463615,6123.765732
BALANCE_FREQUENCY,0.876615,0.955938
PURCHASES,906.713206,12576.984324
ONEOFF_PURCHASES,527.334883,8401.216892
INSTALLMENTS_PURCHASES,379.681008,4175.767432
CASH_ADVANCE,932.129884,6585.291964
PURCHASES_FREQUENCY,0.488319,0.734050
ONEOFF_PURCHASES_FREQUENCY,0.199927,0.505961
PURCHASES_INSTALLMENTS_FREQUENCY,0.362294,0.621513
CASH_ADVANCE_FREQUENCY,0.133695,0.308948


In [55]:
resumen_cluster(-1)

{'BALANCE': 6123.765731891893,
 'BALANCE_FREQUENCY': 0.9559377702702703,
 'PURCHASES': 12576.984324324323,
 'ONEOFF_PURCHASES': 8401.216891891892,
 'INSTALLMENTS_PURCHASES': 4175.7674324324325,
 'CASH_ADVANCE': 6585.291963527026,
 'PURCHASES_FREQUENCY': 0.734049945945946,
 'ONEOFF_PURCHASES_FREQUENCY': 0.5059611756756757,
 'PURCHASES_INSTALLMENTS_FREQUENCY': 0.6215133783783784,
 'CASH_ADVANCE_FREQUENCY': 0.3089475675675676,
 'CASH_ADVANCE_TRX': 19.66216216216216,
 'PURCHASES_TRX': 91.74324324324324,
 'CREDIT_LIMIT': 13079.72972972973,
 'PAYMENTS': 18330.142815243245,
 'MINIMUM_PAYMENTS': 8410.849850527027,
 'PRC_FULL_PAYMENT': 0.2674890405405405,
 'TENURE': 11.635135135135135,
 'cluster_id': -1}